In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df =pd.read_csv("/content/insurance_data.csv")

In [3]:
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [4]:
df.shape

(28, 3)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df[['age', 'affordibility']], df.bought_insurance, test_size=0.2, random_state=25)

In [6]:
X_train.shape, X_test.shape

((22, 2), (6, 2))

In [7]:
# Scale the age feature
X_train_scale = X_train.copy()
X_train_scale['age'] = X_train_scale['age']/100

X_test_scale = X_test.copy()
X_test_scale['age'] = X_test_scale['age']/100

In [8]:
X_train_scale

,age,affordibility
0,0.22,1
13,0.29,0
6,0.55,0
17,0.58,1
24,0.50,1
19,0.18,1
25,0.54,1
16,0.25,0
20,0.21,1
3,0.52,0


In [9]:
# Building a simple neural network model
model = keras.Sequential([
    keras.layers.Dense(1,input_shape = (2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

# kernel_initializer: initializing the weight
# bias_initializer: initializing the bias

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics = ['accuracy']
)

model.fit(X_train_scale, Y_train, epochs = 5000)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 10ms/step - loss: 0.5819 - accuracy: 0.6818
Epoch 1377/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5819 - accuracy: 0.6818
Epoch 1378/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5818 - accuracy: 0.6818
Epoch 1379/5000
1/1 [==============================] - 0s 15ms/step - loss: 0.5818 - accuracy: 0.6818
Epoch 1380/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.5817 - accuracy: 0.6818
Epoch 1381/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.5817 - accuracy: 0.6818
Epoch 1382/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.5816 - accuracy: 0.6818
Epoch 1383/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.5816 - accuracy: 0.6818
Epoch 1384/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5815 - accuracy: 0.6818
Epoch 1385/5000
1/1 [==============================] -

In [10]:
model.evaluate(X_test_scale,Y_test)

In [11]:
X_test_scale

In [12]:
model.predict(X_test_scale)

In [13]:
Y_test

In [14]:
# get the value of weight and bias of the build model
coef, intercept = model.get_weights()
coef, intercept

### **Building model from Scratch**

In [15]:
def sigmoid(x):
  import math
  return 1 / (1 + math.exp(-x))

sigmoid(18)

In [16]:
def prediction_function(age, affordibility):
  weighted_sum = coef[0]*age + coef[1]*affordibility + intercept
  return sigmoid(weighted_sum)


In [17]:
prediction_function(0.47,1)

In [18]:
# Function for log loss that is binary loss entropy
def log_loss(y_true, y_predicted):
  epsilon = 1e-15
  y_predicted_new = [max(i,epsilon) for i in y_predicted]
  y_predicted_new = [min(i, 1-epsilon) for i in y_predicted_new]
  y_predicted_new = np.array(y_predicted_new)
  return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [19]:
def sigmoid_numpy(x):
  return 1/(1+np.exp(-x))

sigmoid_numpy(np.array([12,0,1]))

In [35]:
class myNN:
  def __init__(self):
    self.w1 = 1
    self.w2 = 1
    self.bias = 0

  # Creating a fit method
  def fit(self, X, y, epochs, loss_threshold):
    self.w1, self.w2, self.bias = self.gradient_descent(X['age'],X['affordibility'], y, epochs, loss_threshold)

  # Function for prediction
  def predict(self, X_test):
    weighted_sum = self.w1* X_test['age'] + self.w2* X_test['affordibility'] + self.bias
    return sigmoid_numpy(weighted_sum)

  # Function for model building
  def gradient_descent(self, age, affordibility, y_true, epochs, loss_threshold):
    # loss_threshold used to compare model build using tensorflow with this model

    # w1, w2, bias intitialization
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n =len(age)

    for i in range(epochs):
      weighted_sum = w1* age + w2 * affordibility + bias  # y = w1*x + w2*x +b
      y_predicted = sigmoid_numpy(weighted_sum) # calculating the value of y using sigmoid function

      loss = log_loss(y_true, y_predicted)

      # get the derivative values for w1, w2 and bias
      w1d = (1/n)*np.dot(np.transpose(age), (y_predicted-y_true))
      w2d = (1/n)*np.dot(np.transpose(affordibility), (y_predicted-y_true))

      bias_d = np.mean(y_predicted-y_true)

      # Update the value of w1, w2 and bias
      w1 = w1 - rate * w1d
      w2 = w2 - rate * w2d

      bias = bias - rate * bias_d

      if i%50==0:
        print(f"Epoch: {i}, w1: {w1}, w2: {w2}, Bias: {bias}, loss: {loss}")

      if loss<=loss_threshold:
        print(f"Epoch: {i}, w1: {w1}, w2: {w2}, Bias: {bias}, loss: {loss}")
        break

    return w1, w2, bias


In [36]:
myModel = myNN()
myModel.fit(X_train_scale, Y_train, epochs=501, loss_threshold=0.4631)

Epoch: 0, w1: 0.974907633470177, w2: 0.948348125394529, Bias: -0.11341867736368583, loss: 0.7113403233723417
Epoch: 50, w1: 1.503319554173139, w2: 1.108384790367645, Bias: -1.2319047301235464, loss: 0.5675865113475955
Epoch: 100, w1: 2.200713131760032, w2: 1.2941584023238903, Bias: -1.6607009122062801, loss: 0.5390680417774752
Epoch: 150, w1: 2.8495727769689085, w2: 1.3696895491572745, Bias: -1.986105845859897, loss: 0.5176462164249294
Epoch: 200, w1: 3.443016970881803, w2: 1.4042218624465033, Bias: -2.2571369883752723, loss: 0.5005011269691375
Epoch: 250, w1: 3.982450494649576, w2: 1.4239127329321233, Bias: -2.494377365971801, loss: 0.48654089537617085
Epoch: 300, w1: 4.472179522095915, w2: 1.438787986553552, Bias: -2.707387811922373, loss: 0.4750814640632793
Epoch: 350, w1: 4.917245868007634, w2: 1.4525660781176122, Bias: -2.901176333556766, loss: 0.46561475306999006
Epoch: 366, w1: 5.051047623653049, w2: 1.4569794548473887, Bias: -2.9596534546250037, loss: 0.46293944095888917


In [37]:
coef, intercept

(array([[5.060863 ],
        [1.4086521]], dtype=float32),
 array([-2.913703], dtype=float32))

In [34]:
myModel.predict(X_test_scale)

2     0.705020
10    0.355836
21    0.161599
11    0.477919
14    0.725586
9     0.828987
dtype: float64